1. So, I need a comprehensive set of movies and all the people that worked for it. My way to do it would be to split the df vertically into citing movie info df and cited movie df.

2. Then concatenate those, check for doubles and check if they have the same people working on the team or there are discrepancies.

3. Finally cancel replicates of movies, rows that are double basically.

In [2]:
import os
import pandas as pd

# creating the tables with pandas to interact with our datasets

#name_basics = pd.read_csv("../Dataset/name.basics.tsv" , sep='\t', header=0)
crew_data = pd.read_csv("../datasets/crew_data.csv" , header=0)

citing_movies = crew_data[["tconst_movie", "movie", "movie_country", "year", "titleType", "tconst_cit", "nconst_movie", "category_movie", "job_movie"]]
cited_movies = crew_data[["tconst_cit", "cit", "country_cit", "primaryTitle_cit", "year_cit", "titleType_cit", "nconst_cit", "category_cit", "job_cit"]]
#column_names = list(crew_data.columns.values)
#name_basics[:30]
#print(set(crew_data.category_cit.values))
#citing_movies.describe(include="all")
citing_movies.dtypes

# starting to filter dataset
citing_movies = citing_movies[citing_movies['titleType'] != 'short']
citing_movies['year'] = citing_movies['year'].astype("string")

#function to remove .0 in year column
def remove_dotzero(s):
    return s.replace(".0", "")

citing_movies["year"] = citing_movies["year"].apply(remove_dotzero)

#renaming the columns for mental order
#N.B. because I put inplace = True I don't need to reassing the renamed columns to a new df variable, the original df is being modified on the fly.
citing_movies.rename(columns = {'tconst_movie' : 'movie_ID', 'movie' : 'title', 'nconst_movie' : 'person_ID', 'category_movie'  : 'person_role', 'tconst_cit' : 'cites'}, inplace= True)

#drop titleType column
#citing_movies = citing_movies.drop(['titleType', 'job_movie'], axis=1)

#analysing the ds 
temp =  citing_movies.query("movie_ID == 'tt5611424'")
print(len(temp))
temp

#drop titleType column
citing_movies = citing_movies.drop(['titleType', 'job_movie'], axis=1)

temp

15


,movie_ID,title,movie_country,year,titleType,cites,person_ID,person_role,job_movie
0,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
1,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
2,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
3,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3106201,director,\N
4,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
5,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
6,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
7,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm4097598,writer,based on characters by
8,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3359084,producer,producer
9,tt5611424,#FromJennifer (2017),USA,2017,movie,tt4627224,nm3359084,producer,producer


I realize that the reason why the same movie is repeated so many times is because for every movie also the info about the movie they cite. So for every movie cited there will be three lines for the cited movie director, writer and producer.

I will just remove duplicate rows in the df

In [3]:
print(len(citing_movies))
citing_movies = citing_movies.drop_duplicates()
print(len(citing_movies))

citing_movies

3291419
643826


,movie_ID,title,movie_country,year,cites,person_ID,person_role
0,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
4,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm4097598,writer
8,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3359084,producer
12,tt5611424,#FromJennifer (2017),USA,2017,tt2630300,nm3106201,director
13,tt5611424,#FromJennifer (2017),USA,2017,tt2630300,nm4097598,writer
...,...,...,...,...,...,...,...
3291317,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt0376994,nm0795682,producer
3291371,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm1334526,director
3291377,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0356745,producer
3291383,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0404446,producer


In [4]:
#additional exploring of the df
BeKindRewind =  citing_movies.query("movie_ID == 'tt0799934'")
print(len(BeKindRewind))
BeKindRewind = BeKindRewind.drop_duplicates()
print(len(BeKindRewind))
BeKindRewind.describe(include="all")

citing_movies.describe(include="all")
BeKindRewind.describe(include="all")


4410
4410


,movie_ID,title,movie_country,year,cites,person_ID,person_role
count,4410,4410,4410,4410,4410,4410,4410
unique,1,1,3,1,490,3,2
top,tt0799934,Be Kind Rewind (2008),UK,2008,tt0417148,nm0327273,producer
freq,4410,4410,1470,4410,9,1470,2940


In [5]:
from pprint import pprint

# iterating over columns unique values to see what values are repeated
column_names = list(BeKindRewind.columns.values) #put headers in a list
print(column_names)
'''print(len(BeKindRewind.cites.values))
print(len(set(BeKindRewind.cites.values)))'''

i = 0
while i <= len(column_names)-1:
    print("\n\n all values of the " + column_names[i] + " column are: \n")
    print(BeKindRewind[column_names[i]].values)
    print(set(BeKindRewind[column_names[i]].values))
    print(len(set(BeKindRewind[column_names[i]].values)))
 
    i += 1

['movie_ID', 'title', 'movie_country', 'year', 'cites', 'person_ID', 'person_role']


 all values of the movie_ID column are: 

['tt0799934' 'tt0799934' 'tt0799934' ... 'tt0799934' 'tt0799934'
 'tt0799934']
{'tt0799934'}
1


 all values of the title column are: 

['Be Kind Rewind (2008)' 'Be Kind Rewind (2008)' 'Be Kind Rewind (2008)'
 ... 'Be Kind Rewind (2008)' 'Be Kind Rewind (2008)'
 'Be Kind Rewind (2008)']
{'Be Kind Rewind (2008)'}
1


 all values of the movie_country column are: 

['UK' 'UK' 'UK' ... 'USA' 'USA' 'USA']
{'France', 'UK', 'USA'}
3


 all values of the year column are: 

['2008' '2008' '2008' ... '2008' '2008' '2008']
{'2008'}
1


 all values of the cites column are: 

['tt0417148' 'tt0417148' 'tt0417148' ... 'tt0118147' 'tt0118147'
 'tt0118147']
{'tt0216216', 'tt0083542', 'tt0137523', 'tt0104558', 'tt0068646', 'tt0072308', 'tt0107504', 'tt0117571', 'tt0218553', 'tt0104692', 'tt0310357', 'tt0131369', 'tt0248667', 'tt0266915', 'tt0101326', 'tt0076759', 'tt0118880', '

So, repetitions occur because of:
1. members of crew
2. multiple countries of production

with Be kind rewind, the most frequent movie in our dataframe, whe have:

537 citations * 3 crew members * 3 countries of production = 4833 instances

### Now I proceed to clean the cited movies dataframe:

In [6]:
cited_movies = crew_data[["tconst_cit", "cit", "country_cit", "primaryTitle_cit", "year_cit", "titleType_cit", "nconst_cit", "category_cit", "job_cit"]]

# starting to filter dataset
cited_movies = cited_movies[cited_movies['titleType_cit'] != 'short']
cited_movies['year_cit'] = cited_movies['year_cit'].astype("string")

#function to remove .0 in year column
def remove_dotzero(s):
    return s.replace(".0", "")

cited_movies["year_cit"] = cited_movies["year_cit"].apply(remove_dotzero)

#renaming the columns as the citing_movie df
cited_movies.rename(columns = {'tconst_cit' : 'movie_ID', 'cit' : 'title', 'country_cit' : 'movie_country', 'year_cit' : 'year', 'nconst_cit' : 'person_ID', 'category_cit'  : 'person_role'}, inplace= True)

#N.B this df has no 'cite' column: for the movies that are already present in citing_movies, I can just drop the rows that are repeated (drop duplicates, keep first instance).
# but there might be cited movies that cite no film
#  

#drop titleType column
cited_movies = cited_movies.drop(['titleType_cit', 'job_cit', 'primaryTitle_cit'], axis=1)
print(len(cited_movies))
cited_movies = cited_movies.drop_duplicates()
print(len(cited_movies))

cited_movies

3291419
138489


,movie_ID,title,movie_country,year,person_ID,person_role
0,tt4627224,2 Jennifer (2016),USA,2016,nm4097598,writer
1,tt4627224,2 Jennifer (2016),USA,2016,nm1735659,producer
2,tt4627224,2 Jennifer (2016),USA,2016,nm4259291,producer
3,tt4627224,2 Jennifer (2016),USA,2016,nm3106201,producer
12,tt2630300,To Jennifer (2013),USA,2013,NaN,NaN
...,...,...,...,...,...,...
3290984,tt3315342,Logan (2017),Australia,2017,nm0003506,director
3290985,tt3315342,Logan (2017),Australia,2017,nm0291082,writer
3290986,tt3315342,Logan (2017),Australia,2017,nm0338169,writer
3290987,tt3315342,Logan (2017),Australia,2017,nm0739661,writer


In [7]:
cited_movies.describe(include="all")

,movie_ID,title,movie_country,year,person_ID,person_role
count,138489,138489,138450,138489,137832,137832
unique,31615,30881,134,117,44214,3
top,tt0114863,Control (2007),USA,2006,nm0000636,writer
freq,54,60,58935,2827,122,63462


Now I need to find the intersection between citing_movies id and cited_movies.
I need to:
1. find out what movies in cited_movies are already present in citing_movies
2. check wether the information for same movie_ID are coherent
3. if they are, they are duplicates and I can just drop the cited_movie instance
4. else, inquire why the related info are different and eventually assing the cites value to the cited_movie instance.
5. for movies only present in the cited_movie df, leave them in the concatenated df with nan value in cites column

In [8]:
# First I want to save data about citations from the first dataframe
# I will do it just by creating a subdataframe with the relevant columns

citations_data = citing_movies[["movie_ID", "cites"]]
citations_data

,movie_ID,cites
0,tt5611424,tt4627224
4,tt5611424,tt4627224
8,tt5611424,tt4627224
12,tt5611424,tt2630300
13,tt5611424,tt2630300
...,...,...
3291317,tt6565702,tt0376994
3291371,tt6565702,tt1270798
3291377,tt6565702,tt1270798
3291383,tt6565702,tt1270798


In [9]:
# there are 16379 citing movies, 21453 cited movies and 5349 are in common but ofc in the dataframes are repeated
# Than I add cites column to cited movies
#cited_movies.insert(4, 'cites', pd.Series(dtype="string"))

# concatenating
concat_df = pd.concat([citing_movies, cited_movies])
#len(concat_df) #433829
concat_df.drop_duplicates(inplace=True)
#subset=['movie_ID', 'title', 'movie_country', 'year', 'person_ID', 'person_role']

len(concat_df) #142818


782315

Now I need to add the info about the people that I can retreive from name.basics.csv

In [11]:
people_data = pd.read_csv("../datasets/name.basics.tsv", sep="\t", header=0)
people_data = people_data[["nconst", "primaryName", "birthYear", "deathYear"]]
people_data

network2data = concat_df.merge(people_data, left_on="person_ID", right_on="nconst", how="left") #should be 142818 rows still
network2data.to_csv("../datasets/network2data.csv")
